In [1]:
from warnings import simplefilter
simplefilter('ignore')

import numpy as np

In [2]:
def generate_initial_weights(n_origin_neurons, n_destination_neurons):
    return np.random.random(size=(n_destination_neurons, n_origin_neurons))
    
def generate_initial_biases(n_destination_neurons):
    return np.random.random(size=(n_destination_neurons, 1))

def combine_all_layers(input_layer, hidden_layers, output_layer):
    return np.concatenate((input_layer,
                            hidden_layers, 
                            output_layer))

def generate_all_initial_weights(all_layers):
    return [generate_initial_weights(n_origin_neurons, n_destination_neurons) for n_origin_neurons, n_destination_neurons in zip(all_layers[:-1], all_layers[1:])]
    
def generate_all_initial_biases(all_layers):
    return [generate_initial_biases(n_destination_neurons) for n_destination_neurons in all_layers[1:]]        

def feed_forward__neural_network(weights, biases, feature_data):
    neuron_values = [feature_data]
    for w, b in zip(weights, biases):
        layers = np.matmul(w, neuron_values[-1]) + b
        neuron_values.append(layers)

    return neuron_values

def _error_on_output_layer(target_data, layers):
    return (target_data - layers[-1]) / (2 * len(layers[-1]))

def _error_on_hidden_layers(error_on_upcoming_layer, layer, weights):
    return np.multiply(layer, np.matmul(weights.T, error_on_upcoming_layer))

def _derivative_of_cost_with_respect_to_biases(error_on_current_layer):
    return error_on_current_layer

def _derivative_of_cost_with_respect_to_weights(error_on_current_layer, previous_hidden_layer):
    return np.matmul(error_on_current_layer, previous_hidden_layer.T)

def backward_propagation_neural_network(target_data, layers, weights, biases):
    error_on_layers = [_error_on_output_layer(target_data, layers[-1])]
    for l, w in zip(layers[-2::-1], weights[:0:-1]):
        error_on_layers.append(_error_on_hidden_layers(error_on_layers[-1], l, w))
        
    biases_error = np.array([_derivative_of_cost_with_respect_to_biases(error) for error in error_on_layers], dtype=object)
    weights_error = np.array([_derivative_of_cost_with_respect_to_weights(error, layer) for error, layer in zip(error_on_layers, layers[-2::-1])], dtype=object)

    return (weights_error, biases_error)

def update_weights_biases(alpha, weights, weights_error, biases, biases_error):    
    weights = [w + (alpha * w_e) for w, w_e in zip(weights, weights_error[::-1])]
    biases = [b + (alpha * b_e) for b, b_e in zip(biases, biases_error[::-1])]
    
    return (weights, biases)

def train_determinsitic_neural_network(input_layer, hidden_layers, output_layer, feature_data, target_data, alpha, n_iter):
    all_layers = combine_all_layers(input_layer, hidden_layers, output_layer)
    weights = generate_all_initial_weights(all_layers)
    biases = generate_all_initial_biases(all_layers)
    
    for _ in range(n_iter):
        neuron_values = feed_forward__neural_network(weights, biases, feature_data)
        print(neuron_values[-1])
        weights_error, biases_error = backward_propagation_neural_network(target_data, neuron_values, weights, biases)
        weights, biases = update_weights_biases(0.01, weights, weights_error, biases, biases_error)

    return

In [3]:
input_layer = [1]
hidden_layers = [2, 3]
output_layer = [1]

feature_data = np.array([[1]])
target_data = np.array([[2]])

In [4]:
train_determinsitic_neural_network(input_layer, hidden_layers, output_layer, feature_data, target_data, 0.01, 20)

[[1.02864932]]
[[1.04388684]]
[[1.05902143]]
[[1.07405172]]
[[1.08897629]]
[[1.10379367]]
[[1.11850238]]
[[1.13310087]]
[[1.14758761]]
[[1.16196101]]
[[1.17621948]]
[[1.19036143]]
[[1.20438522]]
[[1.21828925]]
[[1.23207189]]
[[1.24573153]]
[[1.25926655]]
[[1.27267538]]
[[1.28595642]]
[[1.29910813]]
